In [1]:
import requests
import sqlite3
import pandas as pd

In [ ]:
### Deze notebook is voor het lezen van de data vanuit de local database en het beantwoorden van de data analyse vragen

In [ ]:
# Fetch data from database
db_path = "/Users/jinhan.bai/Software/personal/skilltest-zypp/skilltest-buienradar/database/weather.db"
conn = sqlite3.connect(db_path)
conn.execute("PRAGMA foreign_keys = ON")
print(pd.read_sql("SELECT COUNT(*) AS n FROM stations", conn))
print(pd.read_sql("SELECT COUNT(*) AS n FROM measurements", conn))

df_meas = pd.read_sql(
    """
    SELECT
        *
    FROM measurements m
    """,
    conn
)
df_station = pd.read_sql(
    """
    SELECT DISTINCT
        *
    FROM stations s
    """,
    conn
)

df_joined = pd.merge(df_meas, df_station, on='stationid', how='outer')
conn.close()


    n
0  39
    n
0  39


In [ ]:
df_joined.head()

,stationid,timestamp,temperature,groundtemperature,feeltemperature,windgusts,windspeedBft,humidity,precipitation,sunpower,measurementid,stationname,lat,lon,regio
0,6215,2025-09-01T13:30:00,21.3,23.3,21.3,10.2,4.0,56.0,0.0,671.0,6215_2025-09-01T13:30:00,Meetstation Voorschoten,52.12,4.43,Voorschoten
1,6225,2025-09-01T13:30:00,NaN,NaN,NaN,10.8,5.0,NaN,NaN,NaN,6225_2025-09-01T13:30:00,Meetstation IJmuiden,52.47,4.57,IJmuiden
2,6229,2025-09-01T13:30:00,NaN,NaN,NaN,10.8,5.0,NaN,NaN,NaN,6229_2025-09-01T13:30:00,Meetstation Texelhors,53.00,4.75,Texel
3,6235,2025-09-01T13:30:00,20.4,22.7,20.4,11.4,5.0,66.0,0.0,858.0,6235_2025-09-01T13:30:00,Meetstation Den Helder,52.92,4.78,Den Helder
4,6239,2025-09-01T13:30:00,18.5,NaN,18.5,9.4,4.0,86.0,NaN,NaN,6239_2025-09-01T13:30:00,Meetstation Zeeplatform F-3,54.85,4.73,Noordzee


In [16]:
# Analyse (op basis van metingen op 1 tijdstip)

## Weather station with highest recorded temperature 
high_temp = df_joined[df_joined['temperature'] == max(df_joined['temperature'])]
print(f'(stationame, temperature) for station with highest measured temperature: {high_temp.stationname, high_temp.temperature}')

## Average temperature van alle metingen
avg_temp = df_joined['temperature'].mean()
print(f'average temperature across all stations: {avg_temp}')

## Station with biggest difference between feel and actual temperature
df_joined['diff_temp'] = abs(df_joined['feeltemperature'] - df_joined['temperature'])
diff_max = df_joined[df_joined['diff_temp'] == max(df_joined['diff_temp'])]
print(f'(stationname, temperature difference) for station with biggest difference: {diff_max.stationname, diff_max.diff_temp}')

## Weather station in North sea (blijft constant)
noordzee = df_station[df_station['regio'] == "Noordzee"]
print(f'Stations located in regio Noordzee: {noordzee.stationname}') 

(stationame, temperature) for station with highest measured temperature: (33    Meetstation Gilze Rijen
Name: stationname, dtype: object, 33    23.6
Name: temperature, dtype: float64)
average temperature across all stations: 20.258823529411767
(stationname, temperature difference) for station with biggest difference: (27    Meetstation Goes
Name: stationname, dtype: object, 27    2.9
Name: diff_temp, dtype: float64)
Stations located in regio Noordzee: 38    Meetstation Zeeplatform F-3
Name: stationname, dtype: object
